## Data description

### explanation of data (file it came from, which response is being predicted, which sequences)
The DNA sequences that we used are from "concatenated.fasta". This file is generated as explained 
[here](https://github.com/solislemuslab/dna-nn/blob/master/notebook.md) with OG_X.aligned.fasta files, one per OG.

The result/response file that we use used is Perron_predictAMR_2021_updated.xlsx, received in Dec. 6th, 2021 by Email. We used all 8 parameters for regression. We processed the new response with this [jupyter notebook](https://github.com/solislemuslab/dna-nn/blob/master/nn-methods/yuren/new-data-process.ipynb) to csv files before training models on CHTC server.

### data encoding (how are sequences encoded)

The data are first encoded into DNA words (3 nucleotides as one unit), which is done before in DNA-NN-Theory as inspired by this [paper](http://dx.doi.org/10.4236/jbise.2016.95021). For examnple, the input sequence 'ATGCA', will be eoncoded into words 'ATG', 'TGC', 'GCA' and each word will be represented by an integer. This will encode the DNA sequences with length N into the word sequence with length N-2. Then, we use one-hot encoding on the resulting word sequence the same as [DNA-NN-Theory paper](https://arxiv.org/abs/2012.05995).



### data splitting

We randomly splitted the data to use 70% of dataset for training and remaining 15% for validating and 15% for testing.


### data augmentation and simulation

We augmented the data by simulating sequences with iqtree2 from concatenated.fasta with commend "iqtree2 --alisim alignment_mimic -s path-to-concatenated.fasta." This will doulbe our sample size. Then, we augmented the data with reverse complement, which is inspired by this [paper](https://doi.org/10.1016/j.ab.2021.114120), to double the sample size again.

We augment the data with simulated sequences, we generate response data (output of the model) by randomly selecting from the seperated distribution. More specifically, we seperate the original responses into 2 class (less susceptible or more susceptible) with data in tab "binarized succeptibility key", compute mean and sd for both two classes seperatedly, and select randomly from the normal distribution seperatedly (e.g. if the original response is from less susceptible group, we select ramdonly from the distribution computed by responses that are less susceptable for the simulated sequence). 

Besides, we tried not using any data augmentation and bootstraping as inspired by this [paper](https://www.pnas.org/content/93/23/13429). However, these do not work for our model (the training loss did not decrease at all, which means that the model was not albe to be trained).

## Model characteristics

### NN architecture

We adapted from the model cnn_nguyen_conv1d_2_conv2d from the paper for DNA-NN-Theory and [github page](https://github.com/solislemuslab/dna-nn-theory/blob/master/cnn/dna_nn/model.py#L23).

To modified to model for regression, we change the final layer to a linear dense layer, i.e., from Dense(1, activation='sigmoid') to Dense(1, activation="linear").

### when was learning stopped? Early stopped or fixed number of epochs

During the training process, we set the number of training epoches to 300. We also set earlystopping with patience as 50 epoches on validation loss so that the process will end with 50 epoches without reduction on validation loss (reducing validation loss means that the model is still learning from the training data and valudation results are closer to the expected output. At the end of each epoch, we validate the model and save the model if the validation loss is lower than those of all the epoches before so that we only save the best model. The model usually stop after trainig for around 100 epoches.

### Optimizer used, tuning parameters

We used the adam optimizer and default learning rate (0.001).

## Results (TODO)

The models were trained on CHTC servers with the scripts [here](https://github.com/solislemuslab/dna-nn/tree/master/nn-methods/yuren/CHTC-scripts/cnn)